# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Convert ALOS-2 Data to GCOV

In preparation for NISAR, this notebook does:
- ALOS-2 L1.1 (must be StripMap) to NISAR RSLC with https://github.com/isce-framework/isce3/blob/develop/share/nisar/examples/alos2_to_nisar_l1.py
- NISAR DEM staging with https://github.com/isce-framework/isce3/blob/develop/python/packages/nisar/workflows/stage_dem.py 
- NISAR RSLC to GCOV with  https://github.com/isce-framework/isce3/blob/develop/python/packages/nisar/workflows/gcov.py


Requirements:
- a CPU or GPU instance in the On-Demand System 
- updated ISCE3 repo and ISCE3 conda environment with v0.22.0


for questions/feedback, contact Alex Christensen

<a id="TOC"></a>
### Table of Contents
1.  [Step 1: Setup](#SEC_1)
2.  [Step 2: Download ALOS-2 files](#SEC_2)
3.  [Step 3: ALOS-2 SLC to NISAR RSLC](#SEC_3)
4.  [Step 4: Generate DEM](#SEC_4)
5.  [Step 5: NISAR RSLC to GCOV](#SEC_5)
6.  [Step 6: Figures](#SEC_6)

<a id="SEC_1"></a>

# Step 1: Setup


## Import Packages

In [ ]:
## Import packages

import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template
from osgeo import gdal
import numpy as np
import math
import boto3
import shutil
import matplotlib.pyplot as plt
import imageio
import asf_search as asf
from shapely import Polygon
import json
import glob
import pandas as pd

from yaml import safe_load, safe_dump





## Define Functions

Function to configure GCOV

In [ ]:
### If you want to change to UTM coordiates, set utm = True and choose the x and y posting. The default is 20m
def setup_gcov_runconfig(fid,DEM,EPSG,uly='',ulx='',lry='',lrx=''):
    with open('/home/jovyan/isce3/share/nisar/defaults/gcov.yaml') as f:
        doc = safe_load(f)
    folder = '_'.join(fid.split('_')[:18])
    doc['runconfig']['groups']['input_file_group']['input_file_path']=  f"{RSLC_dir/folder.replace('GCOV','RSLC')/folder.replace('GCOV','RSLC')}.h5"
    doc['runconfig']['groups']['product_path_group']['sas_output_file']=  f"{GCOV_dir/fid/fid}.h5"

    doc['runconfig']['groups']['dynamic_ancillary_file_group']['dem_file'] = f"{DEM_dir/DEM}.vrt"
    doc['runconfig']['groups']['product_path_group']['scratch_path'] = f"{TMP_dir}/"
    doc['runconfig']['groups']['processing']['geocode']['output_epsg'] = EPSG
    doc['runconfig']['groups']['processing']['geocode']['output_posting']['A']['x_posting'] = x_posting
    doc['runconfig']['groups']['processing']['geocode']['output_posting']['A']['y_posting'] = y_posting
    doc['runconfig']['groups']['processing']['geocode']['save_rtc_anf_gamma0_to_sigma0']= True
    doc['runconfig']['groups']['processing']['rtc']['input_terrain_radiometry'] = f"sigma0" ##f"beta0"
    
    # doc['']['groups']['processing']['geocode']['save_layover_shadowmask'] = True
    # doc['runconfig']['groups']['processing']['rtc']['save_incidence_angle'] = True 
    # doc['runconfig']['groups']['processing']['rtc']['save_local_inc_angle'] = True 
    if uly != '':
        doc['runconfig']['groups']['processing']['geocode']['top_left']['y_abs'] = uly
        doc['runconfig']['groups']['processing']['geocode']['top_left']['x_abs'] = ulx
        doc['runconfig']['groups']['processing']['geocode']['bottom_right']['y_abs'] = lry
        doc['runconfig']['groups']['processing']['geocode']['bottom_right']['x_abs'] = lrx
        # doc['runconfig']['groups']['processing']['geocode']['y_snap'] = ''
        # doc['runconfig']['groups']['processing']['geocode']['x_snap'] = ''
        print(uly,ulx,lry,lrx)
    print('yaml saved to %s' %(GCOV_dir/fid/(fid + '.yaml')))
    Path(GCOV_dir/fid).mkdir(parents=True, exist_ok=True)
    with open(GCOV_dir/fid/(fid + '.yaml'), 'w') as f:
        safe_dump(doc, f, default_flow_style=False)
            



## Set directories

Files will be stored in ***/home/jovyan***. They will be copied to the ***nisar-st-data-ondemand*** S3 bucket at the end of the notebook with the option to delete local copies.

In [ ]:
notebook_dir = Path(os.getcwd())
# working_dir = Path('/home/jovyan') 
working_dir = Path('/scratch/alex_eco_test/')
## if too many files, use scratch
# working_dir = Path('/scratch/<yourfolder>/') 

HOME_DIR = os.environ['HOME']
output = !nvcc --version
if output == ['/bin/bash: nvcc: command not found']:
    print('running CPU instance')
    ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')
    isce3_env = 'isce3_src_cpu'
else:
    print('running GPU instance')
    ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')
    isce3_env = 'isce3'

# jeff_path = '/scratch/jpon/scripts/DAAC_Ondemand_Reformat/renameH5Product_v2.py'
jeff_path = '/scratch/alex_eco_test/scripts/renameH5Product_v3.py'

## make sure isce3 version is correct  
!source activate {isce3_env} && python -c "import isce3; print(isce3.__version__); "

Files are moved to the ***nisar-st-data-ondemand*** S3 bucket:
- ALOS-1 zip files are saved to ***/ALOS1_zip/*** 
- All processed files (L0B, RSLC, GCOV, DEMs) are saved to ***/ALOS1_processed/***

In [ ]:
bucket_name = "nisar-st-data-ondemand"
processed_bucket = 'ALOS2_processed'
zip_bucket = 'ALOS2_zip/'
s3 = boto3.client("s3")
response = s3.list_objects_v2(
            Bucket=bucket_name,
            Prefix = zip_bucket)
contents = response.get('Contents')

existing_ALOS2_zipfiles = [contents[i].get('Key').split('/')[-1] for i in range(len(contents)) if '.zip' in contents[i].get('Key')]
existing_ALOS2_zippaths = [contents[i].get('Key') for i in range(len(contents)) if '.zip' in contents[i].get('Key')]


## Choose your AOI

New directories will be made to store files for this AOI.

For ALOS2, AOI names must match the "Request" name in the ALOS2 ASF Spreadsheet


In [ ]:
# aoi = 'Wax Lake Delta_Louisiana'
# sheet = 'Wetlands (L1.1)'
# sheet = 'Hazards (L1.1)'

aoi = 'BONA'
sheet = 'Forest (L1.1)'

aoi_str = aoi.replace(" ", "_")
aoi_dir = working_dir/aoi_str


<a id="SEC_2"></a>
# Step 2: Download ALOS-2 Files


## Get the ALOS2 URLs 

If you install **openpyxl**  to your isce3 environment, you can search the ALOS2 spreadsheed directly. Otherwise, you need to copy the list of URLS manually

The cell below will ask you which path and frame you want to process (if there are more than one available for your area of interest). The options will be listed in the prompt


In [ ]:
## Open ALOS-2 spreadsheed, this file is static. If you're looking for the latest ALOS-2 acquisitions, you will need to download the latest spreadsheet
xlsx = pd.ExcelFile('/scratch/alex_eco_test/ALOS-2_Restricted_Available_at_ASF_20240604.xlsx')
df = pd.read_excel(xlsx,sheet)

## Filter the ALOS2 data using name, path, frame, etc
df_filtered = df[df['Request']==aoi]

## Filter if more than one path
paths = df_filtered.groupby('Path').size()
path = int(input('Which path to process? %s' %(paths.index.values)))
df_filtered = df_filtered[df_filtered['Path']==path]

## Filter if more than one frame
frames = df_filtered.groupby('Frame').size()
frame = int(input('Which frame to process? %s' %(frames.index.values)))
df_filtered = df_filtered[df_filtered['Frame']==frame]

frame_dir = aoi_dir / str(path) / str(frame) 

## Filter if more than one Beam
beams = df_filtered.groupby('Beam').size()
if len(np.unique(beams.index.values)) >1:
    beam = input('Which Beam to process? %s' %(beams.index.values))
    df_filtered = df_filtered[df_filtered['Beam']==beam]
    frame_dir = aoi_dir / str(path) / str(frame) / str(beam)
    
else:
    beam = beams.index.values[0]
    frame_dir = aoi_dir / str(path) / str(frame) #/ str(beam)
    
print('Working directory for Path %s, Frame %s, Beam %s is %s' %(path, frame, beam, frame_dir))



In [ ]:
list_of_ALOS_SLCs = list(df_filtered['Zip Link'])
list_of_ALOS_SLCs.sort()

print('You will download %s ALOS2 files, which will require at least %s GB. If you do not have enough space, change your working_dir to scratch and rerun the notebook' %(len(list_of_ALOS_SLCs),3.1*len(list_of_ALOS_SLCs)))

Or manually enter the list of URLs to download. You can only process one frame at a time since the workflow assumes all images are in a stack

In [ ]:
## Example using Mount St. Helens

# list_of_ALOS_SLCs = ['https://cumulus.asf.alaska.edu/L1.1/A4/0000436021_001001_ALOS2402350910-211104.zip',
                     # 'https://cumulus.asf.alaska.edu/L1.1/A4/0000437308_001001_ALOS2404420910-211118.zip',
                     # 'https://cumulus.asf.alaska.edu/L1.1/A4/0000461338_001001_ALOS2410630910-211230.zip',
                     # 'https://cumulus.asf.alaska.edu/L1.1/A4/0000461340_001001_ALOS2408560910-211216.zip',
                     # 'https://cumulus.asf.alaska.edu/L1.1/A4/0000461343_001001_ALOS2406490910-211202.zip',
                     # 'https://cumulus.asf.alaska.edu/L1.1/A4/0000538468_001001_ALOS2512060910-231116.zip']

# frame = list_of_ALOS_SLCs[0].split('-')[0][-4:]
# frame_dir = aoi_dir / str(path) / str(frame) #/ str(beam)


## Make local directories for each NISAR product

In [ ]:
ALOS2_dir = aoi_dir / 'ALOS2'
try:
    frame_dir
except:
    frame_dir = aoi_dir
RSLC_dir = frame_dir / 'RSLC'
DEM_dir = frame_dir / 'DEM'
GCOV_dir = frame_dir / 'GCOV'
TMP_dir = frame_dir / 'TMP'

Path(frame_dir).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(GCOV_dir).mkdir(parents=True, exist_ok=True)
Path(TMP_dir).mkdir(parents=True, exist_ok=True)



## Download the ALOS2 files using wget

### This cell will attempt to download and unzip the ALOS-2 SLC files. It will check if the files already exist in S3 or locally and will only download and unzip if necessary.

In [ ]:
ALOS_ids = []

for a in range(len(list_of_ALOS_SLCs)):
    filename =  list_of_ALOS_SLCs[a].split('/')[-1]
    ALOS_ids.append(('.').join(filename.split('.')[:-1]))
    print('Requested File: ', filename)
    print(os.path.isdir(ALOS2_dir/filename[:-4]))
    print(os.path.isfile(ALOS2_dir/filename))
    print((filename in existing_ALOS2_zipfiles))
    if os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4])==True & (filename in existing_ALOS2_zipfiles)==True:
        print('ALOS1 is stored on S3 and already available locally and unzipped')
    elif (os.path.isfile(ALOS2_dir/filename)==False) & (filename in existing_ALOS2_zipfiles):
        i = existing_ALOS2_zipfiles.index(filename)
        s3_path_new = existing_ALOS2_zippaths[i]
        print('\tALOS2 zip is already available at S3 PATH: ', s3_path_new)
        print('\tMove ALOS2 zip from S3 to local')
        s3.download_file(bucket_name, s3_path_new , ALOS2_dir/filename)
        
    elif (os.path.isfile(ALOS2_dir/filename)==False) & (filename not in existing_ALOS2_zipfiles):
        print('\tALOS2 is not available anywhere')
        print('\tDownloading ALOS2 zip ')
        command = f"wget -P {ALOS2_dir} -q {list_of_ALOS_SLCs[a]}"
        output = subprocess.check_output(command, shell=True)
        print('\tMoving a copy ALOS2 zip to S3 bucket')
        print('\tMoving a copy ALOS1 zip to S3 bucket')
        s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(zip_bucket,aoi_str,filename))


    elif (os.path.isfile(ALOS2_dir/filename)==True) & (filename not in existing_ALOS2_zipfiles):
        print('\tALOS2 is available locally, but not on S3')
        print('\tMoving a copy ALOS2 zip to S3 bucket')
        s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(zip_bucket,aoi_str,filename))

    else: 
        print('\tALOS2 zip file exists locally and on S3')
        
    if os.path.isdir(ALOS2_dir/filename[:-4])==False:
        print('\tunzipping the ALOS2 file locally')
        try:
            with zipfile.ZipFile(ALOS2_dir/filename, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/filename[:-4])
        except:
            print('\tALOS2 zip is bad, re-downloading')
            os.remove(ALOS2_dir/filename)
            command = f"wget -P {ALOS2_dir} -q {list_of_ALOS_SLCs[a]}"
            output = subprocess.check_output(command, shell=True)
            print('\t\tMoving a copy ALOS2 zip to S3 bucket')
            s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(zip_bucket,aoi_str,filename))
            with zipfile.ZipFile(ALOS2_dir/filename, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/filename[:-4])
        print('\t\tdeleting the local zip file')
        os.remove(ALOS2_dir/filename) ## comment to remove the alos2 zip file from your local workspace
    

## Get a list of unzipped ALOS-2 files


In [ ]:
all_matching_frame = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(dirnames,'*')]
ALOS2folders = [i for i in all_matching_frame if any(j in i for j in ALOS_ids)]
ALOS2folders.sort()
ALOS2folders

<a id="SEC_3"></a>

# Step 3: ALOS-2 SLC to NISAR RSLC

## Convert ALOS2 SLC to NISAR RSLC format
Uses ***alos2_to_nisar_l1.py***

In [ ]:

for ALOS2folder in ALOS2folders[:]:
    ALOS2_id = ALOS2folder.split('/')[-1]
    print('')
    print(ALOS2_id)
    if (os.path.isfile(f"{RSLC_dir/ALOS2_id/ALOS2_id}_rslc.h5")==True) | len(glob.glob(str(RSLC_dir /'*'/ ('*RSLC*20%s*.h5' %(ALOS2_id.split('-')[-1])))))>0:
        print('NISAR RSLC already converted')
    else:
        Path(RSLC_dir/ALOS2_id).mkdir(parents=True, exist_ok=True)

        !source activate {isce3_env} && python {ISCE3_BUILD_DIR}/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id/ALOS2_id}_rslc.h5
        

## Rename RSLCs to NISAR naming conventions

In [ ]:
all_RSLCs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(RSLC_dir)
                for f in fnmatch.filter(files,'*_rslc.h5')]
all_RSLCs.sort()
for NISAR_RSLC in all_RSLCs[:]:
    print('')
    print(NISAR_RSLC)
    path = os.path.dirname(NISAR_RSLC) + '/'
    !source activate {isce3_env} && python {jeff_path} -f {NISAR_RSLC} -o {path}/
    !rm -r {NISAR_RSLC}
    


In [ ]:
all_RSLCs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(RSLC_dir)
                for f in fnmatch.filter(files,'*RSLC*.h5')]
all_RSLCs.sort()


## Choose reference image from stack of RSLCs 
The first image in the stack will be the reference image. The DEM and extent of final GCOV files will be based on this reference image

In [ ]:
ALOS2_ref = all_RSLCs[0].split('/')[-1].split('.')[0]
print('')
print('Reference Image will be :' ,ALOS2_ref)


<a id="SEC_4"></a>

# Step 4: Generate DEM

## Get a DEM for the first reference image

Uses ***stage_dem.py***

In [ ]:
if os.path.isfile(f"{DEM_dir/ALOS2_ref}.vrt")==False:
    command = f'source activate {isce3_env} && python {ISCE3_BUILD_DIR}/packages/nisar/workflows/stage_dem.py -p {RSLC_dir/ALOS2_ref/ALOS2_ref}.h5 -o {DEM_dir/ALOS2_ref}.vrt'     
    subprocess.check_output(command, shell=True)

DEM = ALOS2_ref

In [ ]:
x_posting = ''
y_posting = ''
EPSG = ''
### If you want to change to UTM coordiates, set utm = True and choose the x and y posting. The default is 20m
utm = True
if utm:
    src = gdal.Open(f"{DEM_dir/ALOS2_ref}.vrt")
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)


    x1,y1,x2,y2 = math.floor(ulx),math.floor(uly),math.floor(lrx),math.floor(lry)
    zone = int(np.ceil((ulx + 180)/6))

    if y1>=0:
        EPSG = 32600+zone
    elif y1<0:
        EPSG = 32600+zone

    x_posting = 20 ## choose posting
    y_posting = 20 ## choose posting
    
    
print('EPSG: ', EPSG)



<a id="SEC_5"></a>

# Step 5: NISAR RSLC to GCOV

## Run the GCOV processor on the reference image

This image will be used to determine the bounding box of the remaining images in the stack

Uses ***gcov.py***

In [ ]:
ALOS2_ref2 = ALOS2_ref.replace('RSLC','GCOV')

print(ALOS2_ref2)
setup_gcov_runconfig(ALOS2_ref2,DEM,EPSG)
!source activate {isce3_env} && python {ISCE3_BUILD_DIR}/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_ref2/ALOS2_ref2}.yaml

NISAR_GCOV = glob.glob(str(GCOV_dir / ALOS2_ref2 / '*GCOV*.h5'))[0]

!source activate {isce3_env} && python {jeff_path} -f {NISAR_GCOV} -o {GCOV_dir/ALOS2_ref2}/
os.remove(NISAR_GCOV)
NISAR_GCOV = glob.glob(str(GCOV_dir / ALOS2_ref2 / '*GCOV*.h5'))[0]
ALOS2_ref_new = NISAR_GCOV.split('/')[-1].split('.')[0]
shutil.move(GCOV_dir / ALOS2_ref2, GCOV_dir / ALOS2_ref_new)

## Get top left coordinates of bottom right coordinates of reference GCOV

In [ ]:
f = h5py.File(f"{GCOV_dir/ALOS2_ref_new/ALOS2_ref_new}.h5" , "r") 
a_group_key = list(f.keys())[0]
ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object

ulx = x_posting * round(ds_x[0]/x_posting)
lrx = x_posting * round(ds_x[-1]/x_posting)
uly = y_posting * round(ds_y[0]/y_posting)
lry = y_posting * round(ds_y[-1]/y_posting)

print('Force top left to be: %s %s' %(ulx,uly))
print('Force top right to be: %s %s' %(lrx,lry))



## Process the remaining RSLC files to GCOV using the bounding box set by the reference GCOV

### Also generates the metadata cube layers

Uses ***gcov.py***

In [ ]:
for ALOS2_RSLC in all_RSLCs[:]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1].split('.')[0].replace('RSLC','GCOV')
    if crop:
        ALOS2_id = ALOS2_id + '_crop'
    print('\n\n\n')
    print(ALOS2_id)

    if os.path.isfile(f"{GCOV_dir/ALOS2_id/ALOS2_id}.h5")==False:
        GCOV_dir/ALOS2_id/ALOS2_id
        ## GCOV processor
        print('\n>>>>>>>gcov.py')
        setup_gcov_runconfig(ALOS2_id,DEM,EPSG,uly,ulx,lry,lrx)
        !source activate {isce3_env} && python {ISCE3_BUILD_DIR}/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_id/ALOS2_id}.yaml

        NISAR_GCOV = glob.glob(str(GCOV_dir / ALOS2_id / '*GCOV*.h5'))[0]
        
        !source activate {isce3_env} && python {jeff_path} -f {NISAR_GCOV} -o {GCOV_dir/ALOS2_id}/
        os.remove(NISAR_GCOV)
        NISAR_GCOV = glob.glob(str(GCOV_dir / ALOS2_id / '*GCOV*.h5'))[0]
        ALOS2_id_new = NISAR_GCOV.split('/')[-1].split('.')[0]
        shutil.move(GCOV_dir / ALOS2_id, GCOV_dir / ALOS2_id_new)
        shutil.move(GCOV_dir / ALOS2_id_new / (ALOS2_id + '.yaml'), GCOV_dir / ALOS2_id_new / (ALOS2_id_new+ '.yaml'))
        
        ## Get metadata cubes
        print('\n\n>>>>>>>get_product_geometry.py')
        !source activate {isce3_env} && python {ISCE3_BUILD_DIR}/packages/nisar/workflows/get_product_geometry.py --dem {DEM_dir/DEM}.vrt --od {GCOV_dir/ALOS2_id_new} {NISAR_GCOV}
    else:
        print('GCOV already processed')
        
        

<a id="SEC_6"></a>

# Step 6: Figures

## Make Figures

In [ ]:
all_GCOVs = glob.glob(str(GCOV_dir/'**'/'*GCOV*.h5'),recursive=True)
if crop:
    all_GCOVs = [item for item in all_GCOVs if 'crop' in item]
else:
    all_GCOVs = [item for item in all_GCOVs if 'crop' not in item]
all_GCOVs.sort()

for NISAR_GCOV in all_GCOVs[:]:
    ALOS2_id = NISAR_GCOV.split('/')[-1][:-3]
    
    print('')
    print(ALOS2_id)
    f = h5py.File(f"{GCOV_dir/ALOS2_id/ALOS2_id}.h5", "r") 
    a_group_key = list(f.keys())[0]
    ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
    ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object
    HH = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HHHH'][()] 
    HV = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HVHV'][()] 
    print('Width x Height:' , ds_x.shape, ds_y.shape)
    # ulx = x_posting * round(ds_x[0]/x_posting)
    # lrx = x_posting * round(ds_x[-1]/x_posting)
    # uly = y_posting * round(ds_y[0]/y_posting)
    # lry = y_posting * round(ds_y[-1]/y_posting)
    extent= (ds_x[0],ds_y[0],ds_x[-1],ds_y[-1]) 
    print('Top Left: ', ds_x[0],ds_y[0])
    print('Bottom Right: ', ds_x[-1],ds_y[-1])
    
    fig, [ax1,ax2] = plt.subplots(1,2,figsize = (15,10))
    im1 = ax1.imshow(HH, interpolation = 'nearest',
          extent = extent, vmin=0,vmax=0.5,cmap='gray')
    cbar1 = plt.colorbar(im1, ax=ax1,
            shrink=0.5, orientation='horizontal',
            pad=0.1, aspect=50,label='HH')
    im2 = ax2.imshow(HV, interpolation = 'nearest',
          extent = extent, vmin=0,vmax=0.1,cmap='gray')
    cbar2 = plt.colorbar(im2, ax=ax2,
            shrink=0.5, orientation='horizontal',
            pad=0.1, aspect=50,label='HV')
    plt.suptitle(ALOS2_id)
    if crop:
        plt.savefig(GCOV_dir/ALOS2_id/ (ALOS2_id + '_HH-HV_crop.png'))
        # !aws s3 cp {GCOV_dir/ ALOS2_id/(ALOS2_id + '_HH-HV_crop.png')} s3://nisar-st-data-ondemand/{processed_bucket}/{'/'.join(str((GCOV_dir/ALOS2_id/ (ALOS2_id + '_HH-HV_crop.png'))).split('/')[3:])}

    else:
        plt.savefig(GCOV_dir/ALOS2_id/ (ALOS2_id + '_HH-HV.png'))
        # !aws s3 cp {GCOV_dir/ ALOS2_id/(ALOS2_id + '_HH-HV.png')} s3://nisar-st-data-ondemand/{processed_bucket}/{'/'.join(str((GCOV_dir/ALOS2_id/ (ALOS2_id + '_HH-HV.png'))).split('/')[3:])}


    ## Move figure to S3 bucket



## Move all files to S3

In [ ]:
## move entire folder to S3
all_files = [os.path.join(dirpath,f)
            for dirpath,dirnames, files in os.walk(aoi_dir)
            for f in fnmatch.filter(files,'*')]

for file in all_files:
    if ('ALOS2' not in file) & ('TMP' not in file):
        # print(f"s3://nisar-st-data-ondemand/{processed_bucket}/{'/'.join((file).split('/')[3:])}")
        !aws s3 cp {file} s3://nisar-st-data-ondemand/{processed_bucket}/{'/'.join((file).split('/')[3:])}


## Clean TMP files

In [ ]:
# shutil.rmtree(TMP_dir) 
# shutil.rmtree(ALOS2_dir)
shutil.rmtree(aoi_dir)